In [ ]:
import numpy as np
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
import loss_funcs
from evaluate import eval_funcs
from algs import *

from utils.orient import OrientUtils
from evaluate.evaluator import Evaluator
from utils.visualize import SearchPlotter
from utils.image import ImageUtils

import mealpy
import cv2 as cv

plotter = SearchPlotter(update_freq=100)

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/mug/world.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/mug/world_sim.xml", cam_config)

loss_func = loss_funcs.IOU()

alg = UniformSampling(sim_viewer, loss_func=loss_func)
alg_config = UniformSampling.Config(time_limit=200, min_samples=350)

eval_func=eval_funcs.XorDiff(0.1)
evaluator = Evaluator(world_viewer, eval_func=eval_func)

init_location = (0, 1.3, 0.3)
random_orientations = OrientUtils.generate_random(80)
eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]

for pos in eval_positions:
    orig_img, new_pos = world_viewer.get_view_cropped(pos)
    orient, loss = alg.solve(orig_img, new_pos.location, alg_config)
    pred_img, _ = world_viewer.get_view_cropped(ObjectPosition(orient, new_pos.location))

    cv.imshow("Original", orig_img)
    cv.waitKey(0)
    cv.imshow("Predicted", pred_img)
    cv.waitKey(0)

    orig_depth, _ = world_viewer.get_view_cropped(new_pos, depth=True, allow_simulation=False)
    pred_depth, _ = world_viewer.get_view_cropped(ObjectPosition(orient, new_pos.location), depth=True, allow_simulation=False)

    eval_loss = eval_func(orig_depth, pred_depth)
    print(eval_loss)

for alg, config in [(alg, alg_config)]:
    eval_losses = evaluator.evaluate(alg, config, eval_positions)
    print(f"{type(alg).__name__}: {eval_losses}")
